In [1]:
!pip install SpeechRecognition gTTS pyttsx3 wikipedia-api geocoder

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.1 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=242b5b2237bd157f1f32ba0c1c57f49738a63734efd80b1fda4931e048f2e244
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [2]:
from gtts import gTTS
import os

def text_to_speech(text, filename="output.mp3"):
    tts = gTTS(text=text, lang='pt')  # 'pt' pra português
    tts.save(filename)
    os.system(f"mpg123 {filename}")  # Tenta reproduzir o áudio (pode não funcionar no Colab)

# Teste
text_to_speech("Olá, eu sou seu assistente virtual!")

In [3]:
import speech_recognition as sr

def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Diga algo...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio, language='pt-BR')
            print(f"Você disse: {text}")
            return text
        except sr.UnknownValueError:
            print("Não entendi o que você disse.")
            return ""
        except sr.RequestError:
            print("Erro ao conectar com o serviço de reconhecimento.")
            return ""

# Teste (funciona localmente)
# text = speech_to_text()

In [5]:
import wikipediaapi
import webbrowser
import geocoder
import requests

def execute_command(text):
    text = text.lower()

    # Pesquisa na Wikipedia
    if "wikipedia" in text:
        query = text.replace("wikipedia", "").strip()
        wiki = wikipediaapi.Wikipedia(user_agent='my-cool-app/1.0', language='pt')
        page = wiki.page(query)
        if page.exists():
            summary = page.summary[:200]  # Limita o resumo
            text_to_speech(f"Aqui está um resumo sobre {query}: {summary}")
            print(summary)
        else:
            text_to_speech("Desculpe, não encontrei nada sobre isso na Wikipedia.")

    # Abrir YouTube
    elif "youtube" in text:
        text_to_speech("Abrindo o YouTube...")
        webbrowser.open("https://www.youtube.com")

    # Localizar farmácia
    elif "farmácia" in text:
        g = geocoder.ip('me')  # Geolocalização por IP
        lat, lng = g.latlng
        text_to_speech(f"Procurando farmácias perto de você...")

        # Exemplo com Google Places API (requer chave API)
        api_key = "SUA_CHAVE_API"  # Substitua pela sua chave
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius=5000&type=pharmacy&key={api_key}"
        response = requests.get(url).json()

        if response['results']:
            pharmacy = response['results'][0]['name']
            address = response['results'][0]['vicinity']
            text_to_speech(f"Farmácia encontrada: {pharmacy}, localizada em {address}")
            print(f"Farmácia: {pharmacy}, Endereço: {address}")
        else:
            text_to_speech("Nenhuma farmácia encontrada nas proximidades.")

    else:
        text_to_speech("Comando não reconhecido.")

# Teste
execute_command("Pesquisar na Wikipedia sobre Python")

In [6]:
def main():
    text_to_speech("Olá! Como posso ajudar?")
    while True:
        command = speech_to_text()
        if command:
            if "sair" in command.lower():
                text_to_speech("Até mais!")
                break
            execute_command(command)

if __name__ == "__main__":
    # main() # Descomente para rodar o loop principal
    pass